# Webscraping

In [41]:
# import necessary modules
import requests
from bs4 import BeautifulSoup

Webscraping Products
Tvs 
    * Smart
    * Digital
Cookers

In [42]:
# request url
url1 = 'https://www.jumia.co.ke/smart-tvs-2282/'
url2 = 'https://www.jumia.co.ke/digital-tvs/'
url3 = 'https://www.jumia.co.ke/home-cooking-appliances-cookers/'

#list urls
urls = [url1, url2, url3]

Check url status

In [43]:
# Sending GET requests to each URL
responses = [requests.get(url) for url in urls]

# Output the status code of each response
for response in responses:
    print(response.url, response.status_code)

https://www.jumia.co.ke/smart-tvs-2282/ 200
https://www.jumia.co.ke/digital-tvs/ 200
https://www.jumia.co.ke/home-cooking-appliances-cookers/ 200


In [44]:
import requests
from bs4 import BeautifulSoup
import csv

# URL of the page to scrape
url = "https://www.jumia.co.ke/home-cooking-appliances-cookers/"

# Function to scrape all product details from the page
def scrape_product_details(url):
    # Send GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        print("Failed to retrieve the webpage.")
        return []

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all <a> tags with class 'core' that contain product information
    products = soup.find_all('a', class_='core')

    product_details = []

    for product in products:
        # Extract product link (relative URL)
        link = product['href'] if 'href' in product.attrs else None
        # Complete the URL if the link is relative
        link = f"https://www.jumia.co.ke{link}" if link and link.startswith('/') else link
        
        # Extract product name (from the 'name' div)
        name = product.find('h3', class_='name').get_text(strip=True) if product.find('h3', class_='name') else "N/A"
        
        # Extract product price (actual price) from the 'prc' div
        price = product.find('div', class_='prc').get_text(strip=True) if product.find('div', class_='prc') else "N/A"
        
        # Extract old price (if available) from the 'old' div
        old_price = product.find('div', class_='old').get_text(strip=True) if product.find('div', class_='old') else "N/A"
        
        # Extract discount (if available) from the 'bdg _dsct _sm' div
        discount = product.find('div', class_='bdg _dsct _sm').get_text(strip=True) if product.find('div', class_='bdg _dsct _sm') else "N/A"
        
        # Extract item ID (from data-gtm-id attribute)
        item_id = product.get('data-gtm-id', "N/A")
        
        # Extract item brand (from data-gtm-brand attribute)
        item_brand = product.get('data-gtm-brand', "N/A")
        
        # Extract the stars rating (from the 'stars' class)
        stars_rating = product.find('div', class_='stars')
        stars_s = stars_rating.get_text(strip=True) if stars_rating else "N/A"
        
        # Extract reviews (from the 'rev' class)
        reviews = product.find('div', class_='rev')
        rev = reviews.get_text(strip=True) if reviews else "N/A"
        
        # Store all the extracted product details
        product_details.append({
            'href': link,
            'item_name': name,
            'prc': price,
            'old': old_price,
            'bdg_dsct_sm': discount,
            'discount': discount,
            'item_id': item_id,
            'item_brand': item_brand,
            'stars_s': stars_s,
            'rev': rev
        })
    
    return product_details

# Scrape product details from the provided URL
products = scrape_product_details(url)

# Save the product details to a CSV file
with open("jumia_scraped_product_details.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=[
        'href', 'item_name', 'prc', 'old', 'bdg_dsct_sm', 'discount', 'item_id', 'item_brand', 'stars_s', 'rev'
    ])
    writer.writeheader()  # Write the header row
    
    # Write each product's details as a row
    for product in products:
        writer.writerow(product)

print(f"Scraped {len(products)} products and saved them to 'jumia_scraped_product_details.csv'.")


Scraped 50 products and saved them to 'jumia_scraped_product_details.csv'.


In [45]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL of the page to scrape
base_url = "https://www.jumia.co.ke/home-cooking-appliances-cookers/"

# Function to scrape all product details from a specific page
def scrape_product_details_from_page(url):
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage: {url}")
        return []

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all <a> tags with class 'core' that contain product information
    products = soup.find_all('a', class_='core')

    product_details = []

    for product in products:
        # Extract product link (relative URL)
        link = product['href'] if 'href' in product.attrs else None
        # Complete the URL if the link is relative
        link = f"https://www.jumia.co.ke{link}" if link and link.startswith('/') else link
        
        # Extract product name (from the 'name' div)
        name = product.find('h3', class_='name').get_text(strip=True) if product.find('h3', class_='name') else "N/A"
        
        # Extract product price (actual price) from the 'prc' div
        price = product.find('div', class_='prc').get_text(strip=True) if product.find('div', class_='prc') else "N/A"
        
        # Extract old price (if available) from the 'old' div
        old_price = product.find('div', class_='old').get_text(strip=True) if product.find('div', class_='old') else "N/A"
        
        # Extract discount (if available) from the 'bdg _dsct _sm' div
        discount = product.find('div', class_='bdg _dsct _sm').get_text(strip=True) if product.find('div', class_='bdg _dsct _sm') else "N/A"
        
        # Extract item ID (from data-gtm-id attribute)
        item_id = product.get('data-gtm-id', "N/A")
        
        # Extract item brand (from data-gtm-brand attribute)
        item_brand = product.get('data-gtm-brand', "N/A")
        
        # Extract the stars rating (from the 'stars' class)
        stars_rating = product.find('div', class_='stars')
        stars_s = stars_rating.get_text(strip=True) if stars_rating else "N/A"
        
        # Extract reviews (from the 'rev' class)
        reviews = product.find('div', class_='rev')
        rev = reviews.get_text(strip=True) if reviews else "N/A"
        
        # Store all the extracted product details
        product_details.append({
            'href': link,
            'item_name': name,
            'prc': price,
            'old': old_price,
            'bdg_dsct_sm': discount,
            'discount': discount,
            'item_id': item_id,
            'item_brand': item_brand,
            'stars_s': stars_s,
            'rev': rev
        })
    
    return product_details

# Function to get the last page number
def get_last_page_number(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the last page link
    last_page_link = soup.find('a', class_='pg', aria_label='Last Page')
    if last_page_link and 'href' in last_page_link.attrs:
        # Extract the page number from the href
        last_page_url = last_page_link['href']
        last_page_number = int(last_page_url.split('page=')[1].split('#')[0])
        print(f"Last page detected: {last_page_number}")  # Print to confirm
        return last_page_number
    else:
        return 1  # Default to 1 page if no last page is found

# Scrape product details from all pages
def scrape_all_products(base_url):
    # Get the total number of pages
    last_page_number = get_last_page_number(base_url)
    all_products = []

    # Iterate over each page and scrape the products
    for page in range(1, last_page_number + 1):
        print(f"Scraping page {page}...")
        page_url = f"{base_url}?page={page}#catalog-listing"
        products_on_page = scrape_product_details_from_page(page_url)
        all_products.extend(products_on_page)
    
    return all_products

# Scrape all product details from all pages
products = scrape_all_products(base_url)

# Save the product details to a CSV file
with open("jumia_scraped_all_products.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=[
        'href', 'item_name', 'prc', 'old', 'bdg_dsct_sm', 'discount', 'item_id', 'item_brand', 'stars_s', 'rev'
    ])
    writer.writeheader()  # Write the header row
    
    # Write each product's details as a row
    for product in products:
        writer.writerow(product)

print(f"Scraped {len(products)} products and saved them to 'jumia_scraped_all_products.csv'.")


Scraping page 1...
Scraped 50 products and saved them to 'jumia_scraped_all_products.csv'.


In [46]:
import requests
from bs4 import BeautifulSoup

# User-Agent headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_last_page_number(url):
    # Make the request to get the page content
    response = requests.get(url, headers=headers)
    
    # Ensure the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page. Status code: {response.status_code}")
        return 1  # Return 1 if the request failed
    
    soup = BeautifulSoup(response.content, 'html.parser')

    # Debug: Let's print out all the <a> tags to check their contents
    all_links = soup.find_all('a', class_='pg')  # Find all <a> tags with class 'pg'
    print("All <a> tags with class 'pg':")
    for link in all_links:
        print(link)

    # Find the <a> tag with the aria-label 'Last Page' (check its presence)
    last_page_link = soup.find('a', attrs={'aria-label': 'Last Page'})
    
    # Debug: Print out the <a> tag for the last page link if found
    if last_page_link:
        print(f"Last page link found: {last_page_link}")
    else:
        print("No 'Last Page' link found.")

    if last_page_link and 'href' in last_page_link.attrs:
        # Extract the href value
        last_page_url = last_page_link['href']
        print(f"Last page link URL: {last_page_url}")  # Debug print statement
        
        # Extract the page number from the URL (after '?page=' and before '#')
        try:
            # Split the URL to extract the page number
            page_number = last_page_url.split('?page=')[1].split('#')[0]
            last_page_number = int(page_number)  # Convert the extracted page number to integer
            print(f"Last page detected: {last_page_number}")  # Confirm output
            return last_page_number
        except Exception as e:
            print(f"Error extracting last page number: {e}")
            return 1  # Default to 1 if error occurs
    else:
        print("Last page link not found")
        return 1  # Default to 1 page if no last page is found


# Testing the function with the base URL
base_url = "https://www.jumia.co.ke/home-cooking-appliances-cookers/"
get_last_page_number(base_url)


All <a> tags with class 'pg':
<a aria-label="Page 2" class="pg" href="/home-cooking-appliances-cookers/?page=2#catalog-listing">2</a>
<a aria-label="Page 3" class="pg" href="/home-cooking-appliances-cookers/?page=3#catalog-listing">3</a>
<a aria-label="Next Page" class="pg" href="/home-cooking-appliances-cookers/?page=2#catalog-listing"><svg class="ic" height="24" viewbox="0 0 24 24" width="24"><use xlink:href="https://www.jumia.co.ke/assets_he/images/i-icons.da6131d9.svg#arrow-right"></use></svg></a>
<a aria-label="Last Page" class="pg" href="/home-cooking-appliances-cookers/?page=50#catalog-listing"><svg class="ic" height="24" viewbox="0 0 24 24" width="24"><use xlink:href="https://www.jumia.co.ke/assets_he/images/i-icons.da6131d9.svg#last-page"></use></svg></a>
Last page link found: <a aria-label="Last Page" class="pg" href="/home-cooking-appliances-cookers/?page=50#catalog-listing"><svg class="ic" height="24" viewbox="0 0 24 24" width="24"><use xlink:href="https://www.jumia.co.ke/a

50

## Scrape Cookers

In [49]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random

# User-Agent headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to get the last page number
def get_last_page_number(url):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve page. Status code: {response.status_code}")
        return 1  # Default to 1 if the request fails
    
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the 'Last Page' link using its aria-label attribute
    last_page_link = soup.find('a', attrs={'aria-label': 'Last Page'})
    
    if last_page_link and 'href' in last_page_link.attrs:
        last_page_url = last_page_link['href']
        try:
            page_number = last_page_url.split('?page=')[1].split('#')[0]
            return int(page_number)
        except Exception as e:
            print(f"Error extracting last page number: {e}")
            return 1  # Default to 1 if error occurs
    else:
        print("Last page link not found.")
        return 1  # Default to 1 page if no last page is found

# Function to scrape product details from a given URL
def scrape_product_details(url):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage: {url}. Status code: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    products = soup.find_all('a', class_='core')

    product_details = []

    for product in products:
        link = product['href'] if 'href' in product.attrs else None
        link = f"https://www.jumia.co.ke{link}" if link and link.startswith('/') else link
        
        name = product.find('div', class_='name').get_text(strip=True) if product.find('div', class_='name') else product.get('data-gtm-name', "N/A")
        price = product.find('div', class_='prc').get_text(strip=True) if product.find('div', class_='prc') else product.get('data-gtm-price', "N/A")
        discount = product.find('div', class_='bdg _dsct').get_text(strip=True) if product.find('div', class_='bdg _dsct') else "N/A"
        brand = product.get('data-gtm-brand', "N/A")
        img_tag = product.find('img')
        img_url = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else "N/A"
        
        data_id = product.get('data-gtm-id', "N/A")
        data_category = product.get('data-gtm-category', "N/A")
        data_discount = product.get('data-ga4-discount', "N/A")
        data_price = product.get('data-ga4-price', "N/A")
        data_name = product.get('data-ga4-item_name', "N/A")
        
        product_details.append({
            'link': link,
            'name': name,
            'price': price,
            'discount': discount,
            'brand': brand,
            'image_url': img_url,
            'data_id': data_id,
            'data_category': data_category,
            'data_discount': data_discount,
            'data_price': data_price,
            'data_name': data_name
        })
    
    return product_details

# Base URL for the cookers category
base_url = "https://www.jumia.co.ke/home-cooking-appliances-cookers/"

# Get the last page number
last_page = get_last_page_number(base_url)

# Initialize an empty list to store all products
cookers = []

# Iterate through all pages from 1 to the last page
for page_num in range(1, last_page + 1):
    page_url = f"{base_url}?page={page_num}#catalog-listing"
    print(f"Scraping page {page_num}...")
    
    # Scrape the products from the current page
    products = scrape_product_details(page_url)
    cookers.extend(products)  # Add the scraped products to the main list

    # Sleep for a random time between requests to avoid overwhelming the server
    time.sleep(random.uniform(1, 3))

# Save the scraped product details to a CSV file
with open("jumia_scraped_cookers.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=[
        "link", "name", "price", "discount", "brand", "image_url",
        "data_id", "data_category", "data_discount", "data_price", "data_name"
    ])
    writer.writeheader()  # Write the header row
    
    for product in cookers:
        writer.writerow(product)

print(f"Scraped {len(cookers)} products and saved them to 'jumia_scraped_cookers.csv'.")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraped 2

## Scrape SmartTvs

In [48]:
url = (f"https://www.jumia.co.ke/home-cooking-appliances-cookers/?page={x}#catalog-listing")

for itm in range(1,51):


SyntaxError: incomplete input (3316088077.py, line 3)